In [1]:
import sys, os
from pathlib import Path

path = Path(os.getcwd())
sys.path.append(str(path.parent))

In [2]:
# Impliments the BayesByBackprop optimizer for BayesKeras

import os
import math
import logging
import sys

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

from tqdm import tqdm
from tqdm import trange

from BayesKeras.optimizers import optimizer
from BayesKeras.optimizers import losses
from BayesKeras import analyzers
from abc import ABC, abstractmethod

# A dumb mistake on my part which needs to be factored out


def softplus(x):
    return tf.math.softplus(x)


class BayesByBackprop():
    def __init__(self):
        print("This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.")

    # I set default params for each sub-optimizer but none for the super class for pretty obvious reasons
    def compile(self, keras_model, loss_fn, batch_size=64, learning_rate=0.15, decay=0.0,
                epochs=10, prior_mean=-1, prior_var=-1, **kwargs):
        ###############################super().compile()###############################
        self.model = keras_model
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.decay = decay
        self.epochs = epochs
        self.loss_func = loss_fn

        self.log_dir = kwargs.get('log_file', '/tmp/BayesKeras.log')

        self.det = kwargs.get('deterministic', False)
        self.inflate_prior = kwargs.get('inflate_prior', 1)
        self.input_noise = kwargs.get('input_noise', 0.0)
        # 先验、后验都调用prior_generator
        print("===============================进入prior_generator===============================")
        self.prior_mean, self.prior_var = self.prior_generator(
            prior_mean, prior_var)
        self.posterior_mean, self.posterior_var = self.prior_generator(
            prior_mean, prior_var)
        print("===============================退出prior_generator===============================")

        self.train_loss = tf.keras.metrics.Mean(name="train_loss")
        self.valid_loss = tf.keras.metrics.Mean(name="valid_loss")

        # Right now I only have one accessory metric. Will come back and add many later.
        self.train_metric = kwargs.get(
            'metric', tf.keras.metrics.SparseCategoricalAccuracy(name="train_acc"))
        self.valid_metric = kwargs.get(
            'metric', tf.keras.metrics.SparseCategoricalAccuracy(name="valid_acc"))
        self.extra_metric = kwargs.get(
            'metric', tf.keras.metrics.SparseCategoricalAccuracy(name="extra_acc"))

        self.robust_train = kwargs.get('robust_train', 0)
        if(self.robust_train != 0):
            print("BayesKeras: Detected robust training at compilation. Please ensure you have selected a robust-compatible loss")
            self.epochs += 1
        self.epsilon = kwargs.get('epsilon', 0.1)
        self.robust_lambda = kwargs.get('rob_lam', 0.5)
        self.robust_linear = kwargs.get('linear_schedule', True)

        self.attack_loss = tf.keras.losses.SparseCategoricalCrossentropy()

        self.loss_monte_carlo = kwargs.get('loss_mc', 2)
        self.eps_dist = tfp.distributions.Exponential(
            rate=1.0/float(self.epsilon))

        self.acc_log = []
        self.rob_log = []
        self.loss_log = []
        ###############################super().compile()###############################

        # Now we get into the BayesByBackprop specific enrichments to the class
        # Post process our variances to all be stds:
        print("计算后验方差")
        for i in range(len(self.posterior_var)):
            self.posterior_var[i] = tf.math.log(
                tf.math.exp(self.posterior_var[i])-1)
        self.kl_weight = kwargs.get('kl_weight', 1.0)
        self.kl_component = tf.keras.metrics.Mean(name="kl_comp")
        print("BayesKeras: Using passed loss_fn as the data likelihood in the KL loss")

        return self

    def train(self, X_train, y_train, X_test=None, y_test=None):
        ###############################super().train()###############################
        self.N = len(X_train)
        train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(
            100).batch(self.batch_size)  # from_tensor_slices: 切分传入Tensor的第一个维度，生成相应的dataset
        test_ds = tf.data.Dataset.from_tensor_slices(
            (X_test, y_test)).batch(self.batch_size)

        if(self.robust_linear):
            self.max_eps = self.epsilon
            self.epsilon = 0.0
            self.max_robust_lambda = self.robust_lambda

        lr = self.learning_rate
        decay = self.decay
        for epoch in range(self.epochs):
            lrate = self.learning_rate * (1 / (1 + self.decay * epoch))

            # Run the model through train and test sets respectively
            print("===============================开始BBB初始采样===============================")
            for (features, labels) in tqdm(train_ds):
                features += np.random.normal(loc=0.0, scale=self.input_noise, size=features.shape)
                self.posterior, self.posterior_var = self.step(
                    features, labels, lrate)
            print("===============================结束BBB初始采样===============================")
            print("===============================开始model_validate===============================")
            for test_features, test_labels in test_ds:
                self.model_validate(test_features, test_labels)
            print("===============================结束model_validate===============================")

            # Grab the results
            (loss, acc) = self.train_loss.result(), self.train_metric.result()
            (val_loss, val_acc) = self.valid_loss.result(), self.valid_metric.result()
            self.logging(loss, acc, val_loss, val_acc, epoch)

            # Clear the current state of the metrics
            self.train_loss.reset_states(), self.train_metric.reset_states()
            self.valid_loss.reset_states(), self.valid_metric.reset_states()
            self.extra_metric.reset_states()

            if(self.robust_linear):
                self.epsilon += self.max_eps/self.epochs
        ###############################super().train()###############################

    def step(self, features, labels, lrate):
        """
        Initial sampling for BBB
        """
        init_weights = []
        noise_used = []
        for i in range(len(self.posterior_mean)):
            noise = tf.random.normal(shape=self.posterior_var[i].shape,
                                     mean=tf.zeros(self.posterior_var[i].shape), stddev=1.0)
            var_add = tf.multiply(softplus(self.posterior_var[i]), noise)
            #var_add = tf.multiply(self.posterior_mean[i], noise)
            w = tf.math.add(self.posterior_mean[i], var_add)
            noise_used.append(noise)
            init_weights.append(w)
        self.model.set_weights(init_weights)

        # Define the GradientTape context
        # Below we add an extra variable for IBP
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(self.posterior_mean)
            tape.watch(self.posterior_var)  # tape.watch(init_weights)
            predictions = self.model(features)

            if(self.robust_train == 0):
                worst_case = predictions  # cheap hack lol
                loss, kl_comp = losses.KL_Loss(labels, predictions, self.model.trainable_variables,
                                               self.prior_mean, self.prior_var,
                                               self.posterior_mean, self.posterior_var,
                                               self.loss_func, self.kl_weight)
            elif(int(self.robust_train) == 1):
                # Get the probabilities
                logit_l, logit_u = analyzers.IBP(
                    self, features, self.model.trainable_variables, eps=self.epsilon)
                #!*! TODO: Undo the hardcoding of depth in this function
                v1 = tf.one_hot(labels, depth=10)
                v2 = 1 - tf.one_hot(labels, depth=10)
                worst_case = tf.math.add(tf.math.multiply(
                    v2, logit_u), tf.math.multiply(v1, logit_l))

                # Now we have the worst case softmax probabilities
                worst_case = self.model.layers[-1].activation(worst_case)
                # Calculate the loss
                loss, kl_comp = losses.robust_KL_Loss(labels, predictions, self.model.trainable_variables,
                                                      self.prior_mean, self.prior_var,
                                                      self.posterior_mean, self.posterior_var,
                                                      self.loss_func, self.kl_weight,
                                                      worst_case, self.robust_lambda)
            elif(int(self.robust_train) == 2):
                features_adv = analyzers.PGD(
                    self, features, self.attack_loss, eps=self.epsilon, num_models=-1)
                # Get the probabilities
                worst_case = self.model(features_adv)
                # print(predictions[0], worst_case[0])
                # Calculate the loss
                loss, kl_comp = losses.robust_KL_Loss(labels, predictions, self.model.trainable_variables,
                                                      self.prior_mean, self.prior_var,
                                                      self.posterior_mean, self.posterior_var,
                                                      self.loss_func, self.kl_weight,
                                                      worst_case, self.robust_lambda)
        # Get the gradients
        weight_gradient = tape.gradient(loss, self.model.trainable_variables)
        mean_gradient = tape.gradient(loss, self.posterior_mean)
        var_gradient = tape.gradient(loss, self.posterior_var)
        #init_gradient = tape.gradient(loss, init_weights)

        posti_mean_grad = []
        posti_var_grad = []
        # !*! - Make the weight and init gradients the same variable and retest
        for i in range(len(mean_gradient)):
            #weight_gradient[i] = tf.math.add(weight_gradient[i], init_gradient[i])
            weight_gradient[i] = tf.cast(weight_gradient[i], 'float32')
            mean_gradient[i] = tf.cast(mean_gradient[i], 'float32')
            f = tf.math.add(weight_gradient[i], mean_gradient[i])
            posti_mean_grad.append(f)
            v = tf.math.divide(
                noise_used[i], 1+tf.math.exp(tf.math.multiply(self.posterior_var[i], -1)))
            v = tf.math.multiply(v, weight_gradient[i])
            v = tf.math.add(v, var_gradient[i])
            posti_var_grad.append(v)
        #gradients = posti_mean_grad

        # APPLICATION OF WEIGHTS
        new_posti_var = []
        new_posti_mean = []
        for i in range(len(mean_gradient)):
            pdv = tf.math.multiply(posti_var_grad[i], lrate)
            pdm = tf.math.multiply(posti_mean_grad[i], lrate)
            v = tf.math.subtract(self.posterior_var[i], pdv)
            m = tf.math.subtract(self.posterior_mean[i], pdm)
            new_posti_var.append(v)
            new_posti_mean.append(m)

        self.train_loss(loss)
        self.train_metric(labels, predictions)
        #self.train_rob(labels, worst_case)
        self.kl_component(kl_comp)
        self.posterior_mean = new_posti_mean
        self.posterior_var = new_posti_var
        return new_posti_mean, new_posti_var

    def prior_generator(self, means, vars):
        if(type(means) == int and type(vars) == int):
            if(means < 0 or vars < 0):
                model_mean, model_var = self._gen_implicit_prior()
                # for i in range(len(model_var)):
                #    model_var[i] = tf.math.log(tf.math.exp(self.model_var[i])-1)
                return model_mean, model_var
        if(type(means) == int or type(means) == float):
            if(means == -1):
                means = 0.0
            mean_params = [means for i in range(len(self.model.weights))]
            means = mean_params
        if(type(vars) == int or type(vars) == float):
            if(vars == -1):
                vars = 0.0
            var_params = [vars for i in range(len(self.model.weights))]
            vars = var_params
        model_mean = []
        model_var = []
        index = 0.0
        for weight in self.model.weights:
            param_index = math.floor(index/2.0)
            mean_i = tf.math.multiply(
                tf.ones(weight.shape), means[param_index])
            vari_i = tf.math.multiply(tf.ones(weight.shape), vars[param_index])
            model_mean.append(mean_i)
            model_var.append(vari_i)
            index += 1
        return model_mean, model_var

    """
    根据模型每一层的形状生成权重w和偏置b的先验均值分布
    权重w：0
    偏置b：std = math.sqrt(self.inflate_prior/(nin)) 
    """
    def _gen_implicit_prior(self):
        print("BayesKeras: Using implicit prior")
        prior_mean = []
        prior_var = []
        for i in range(len(self.model.layers)):
            try:
                sha = self.model.layers[i].get_weights()[0].shape
                b_sha = self.model.layers[i].get_weights()[1].shape
                if(len(sha) > 2):
                    nin = 1
                    for i in range(len(sha)-1):
                        nin *= sha[i]
                else:
                    nin = sha[0]
                # 标准差
                std = math.sqrt(self.inflate_prior/(nin))
                print(sha, std)
                mean_w = tf.zeros(sha)
                var_w = tf.ones(sha) * std
                mean_b = tf.zeros(b_sha)
                var_b = tf.ones(b_sha) * std
                # 分别加入权重w和偏置b的先验均值分布
                prior_mean.append(mean_w)
                prior_mean.append(mean_b)
                prior_var.append(var_w)
                prior_var.append(var_b)
            except:
                pass
        return prior_mean, prior_var

    def model_validate(self, features, labels):
        # self.model.set_weights(self.sample())
        predictions = self.model(features)
        if(self.robust_train == 1):  # We only check with IBP if we need to
            logit_l, logit_u = analyzers.IBP(
                self, features, self.model.get_weights(), self.epsilon)
            #logit_l, logit_u = analyzers.IBP(self, features, self.model.trainable_variables, 0.0)
            v1 = tf.one_hot(labels, depth=10)
            v2 = 1 - tf.one_hot(labels, depth=10)
            worst_case = tf.math.add(tf.math.multiply(
                v2, logit_u), tf.math.multiply(v1, logit_l))
            worst_case = self.model.layers[-1].activation(worst_case)
            v_loss = self.loss_func(
                labels, predictions, worst_case, self.robust_lambda)
            self.extra_metric(labels, worst_case)
        elif(self.robust_train == 2):
            v_loss = self.loss_func(
                labels, predictions, predictions, self.robust_lambda)
            worst_case = predictions
        elif(self.robust_train == 3 or self.robust_train == 5):  # We only check with IBP if we need to
            logit_l, logit_u = analyzers.IBP(
                self, features, self.model.get_weights(), self.epsilon)
            #logit_l, logit_u = analyzers.IBP(self, features, self.model.trainable_variables, 0.0)
            # print(logit_l.shape)
            v1 = tf.squeeze(tf.one_hot(labels, depth=10))
            v2 = tf.squeeze(1 - tf.one_hot(labels, depth=10))
            #print(v1.shape, v2.shape)
            worst_case = tf.math.add(tf.math.multiply(
                v2, logit_u), tf.math.multiply(v1, logit_l))
            # print(worst_case.shape)
            worst_case = self.model.layers[-1].activation(worst_case)
            # print(worst_case.shape)
            v_loss = self.loss_func(labels, worst_case)
            self.extra_metric(labels, worst_case)
        else:
            v_loss = self.loss_func(labels, predictions)
            worst_case = predictions
        self.valid_metric(labels, predictions)
        self.valid_loss(v_loss)
        #self.valid_rob(labels, worst_case)

    def logging(self, loss, acc, val_loss, val_acc, epoch):
        # Local logging
        if(self.robust_train == 0):
            template = "Epoch {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}"
            print(template.format(epoch+1, loss,
                                  acc,
                                  val_loss,
                                  val_acc))
        else:
            rob = self.extra_metric.result()
            template = "Epoch {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}, rob: {:.3f}, (eps = {:.6f})"
            print(template.format(epoch+1, loss,
                                  acc,
                                  val_loss,
                                  val_acc, rob, self.epsilon))
        log_template = "Epoch: {}, Train: [Loss: {:.3f}, Acc: {:.3f}], Test: [Loss: {:.3f}, Acc: {:.3f}]"
        logging.basicConfig(filename=self.log_dir, level=logging.DEBUG)
        logging.info(log_template.format(
            epoch+1, loss, acc, val_loss, val_acc))

    def save(self, path):
        if not os.path.exists(path):
            os.makedirs(path)
        var = []
        for i in range(len(self.posterior_var)):
            var.append(softplus(self.posterior_var[i]))
        np.save(path+"/mean", np.asarray(self.posterior_mean))
        np.save(path+"/var", np.asarray(var))
        self.model.save(path+'/model.h5')
        model_json = self.model.to_json()
        with open(path+"/arch.json", "w") as json_file:
            json_file.write(model_json)
        print("模型%s保存成功" % path)

D:\Users\T470\Anaconda3\envs\CertifiableBayesianInference\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [4]:
# 划分训练集，测试集
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)

In [5]:
# 构建Sequential模型
model = Sequential()
# 构建Dense隐藏层并添加到模型中
# 添加具有512个神经元的Dense隐藏层，使用relu激活函数
model.add(Dense(512, activation="relu", input_shape=(None, 28*28)))
model.add(Dense(10, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 512)         401920    
                                                                 
 dense_1 (Dense)             (None, None, 10)          5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
dict = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'BBB', 'gpu': '-1'}

# epsilon: the strength of the adversary
eps = dict['eps']
# lamada: the probability of seeing the clean data (lambda = 1.0 means no adversarial data considered, lambda = 0.0 means only adversarial data considered)
lam = dict['lam'] 
# robustness mode, 0:标准 1:IBP 2:PGD
rob = dict['rob'] 
# the variational inference method
optim = dict['opt']
gpu = dict['gpu']
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

In [7]:
inf = 10
learning_rate = 0.45; decay=0.0

if (rob == 0):
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
elif (rob != 0):
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss

In [8]:
opt = BayesByBackprop()

This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.


In [9]:
bayes_model = opt.compile(model, loss_fn=loss, learning_rate=learning_rate, epochs=20, # epochs=20, 
                          batch_size=128, linear_schedule=True,
                          decay=decay, robust_train=rob, c=inf,
                          burn_in=3, steps=25, b_steps=20, epsilon=eps, rob_lam=lam)

###############################进入prior_generator###############################
BayesKeras: Using implicit prior
(784, 512) 0.03571428571428571
(512, 10) 0.04419417382415922
BayesKeras: Using implicit prior
(784, 512) 0.03571428571428571
(512, 10) 0.04419417382415922
###############################退出prior_generator###############################
计算后验方差
BayesKeras: Using passed loss_fn as the data likelihood in the KL loss


In [10]:
import datetime
start = datetime.datetime.now()

bayes_model.train(X_train, y_train, X_test, y_test)

end = datetime.datetime.now()
print('Running time: %s' % (end-start))

###############################开始BBB初始采样###############################


100%|██████████████████████████████| 469/469 [03:16<00:00,  2.39it/s]


###############################结束BBB初始采样###############################
###############################开始model_validate###############################
###############################结束model_validate###############################
Epoch 1, loss: 7.757, acc: 0.838, val_loss: 0.232, val_acc: 0.926
###############################开始BBB初始采样###############################


100%|██████████████████████████████| 469/469 [03:05<00:00,  2.53it/s]


###############################结束BBB初始采样###############################
###############################开始model_validate###############################
###############################结束model_validate###############################
Epoch 2, loss: 6.876, acc: 0.940, val_loss: 0.157, val_acc: 0.949
###############################开始BBB初始采样###############################


...

###############################结束BBB初始采样###############################
###############################开始model_validate###############################
###############################结束model_validate###############################
Epoch 18, loss: 4.248, acc: 0.978, val_loss: 0.099, val_acc: 0.971
###############################开始BBB初始采样###############################


100%|██████████████████████████████| 469/469 [03:26<00:00,  2.27it/s]


###############################结束BBB初始采样###############################
###############################开始model_validate###############################
###############################结束model_validate###############################
Epoch 19, loss: 4.210, acc: 0.978, val_loss: 0.096, val_acc: 0.971
###############################开始BBB初始采样###############################


100%|██████████████████████████████| 469/469 [03:07<00:00,  2.50it/s]


###############################结束BBB初始采样###############################
###############################开始model_validate###############################
###############################结束model_validate###############################
Epoch 20, loss: 4.169, acc: 0.979, val_loss: 0.137, val_acc: 0.958


In [11]:
# Save your approxiate Bayesian posterior
bayes_model.save("%s_FCN_Posterior_%s" % (optim, rob))

D:\Users\T470\Anaconda3\envs\CertifiableBayesianInference\lib\site-packages\ipykernel_launcher.py:375: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
D:\Users\T470\Anaconda3\envs\CertifiableBayesianInference\lib\site-packages\ipykernel_launcher.py:376: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


模型BBB_FCN_Posterior_0保存成功
